In [1]:
from model import Encoder, Decoder, Seq2Seq
from data_loader import *
import pandas as pd
import torch.optim.lr_scheduler as lr_scheduler
from torch import optim
import torch.nn.functional as F
import datetime
import pretty_midi
import glob

In [2]:
import os
import matplotlib
import math
matplotlib.use('Agg')
# matplotlib.use("QtAgg")
import ffmpeg
#conda install -c conda-forge ffmpeg-python

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, writers
plt.rcParams['animation.ffmpeg_path'] = '/home/ilc/anaconda3/bin/ffmpeg'#'/usr/bin/ffmpeg'

import numpy as np
import subprocess as sp
from moviepy.video.io.VideoFileClip import VideoFileClip
from moviepy.audio.io.AudioFileClip import AudioFileClip

from midi2audio import FluidSynth

from torch.autograd import Variable
from sklearn.model_selection import KFold

import itertools

In [3]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [4]:
dataset_name_path = f"./midi_list_symbolic_cross.txt" #f"./midi_list.txt"
dataloader = get_dataloader(dataset_name_path, batch_size=20) #[20, 512, 128], [20, 512, 102]
dataset = MidiMotionDataSet(dataset_name_path)

val_dataset_name_path = f"./midi_list_symbolic_cross.txt" #f"./midi_list_eval.txt"
# val_dataloader = get_val_dataloader(val_dataset_name_path, batch_size=40) #[20, 512, 128], [20, 512, 102]

full_data_path = None
with open("./midi_list_symbolic_cross.txt", "r") as file:
    lines = [line.strip() for line in file]
    full_data_path = np.array(lines)

val_data_read = np.reshape(full_data_path, (11, 10))
# print(val_data_read)

learning_rate = 0.001#0.001

# input_size_encoder = 128 #129 #128
# input_size_decoder = 115 #102 #24
# output_size = 115#102 #24

# encoder_embedding_size = 300
# decoder_embedding_size = 300
enc_dropout = 0.5
dec_dropout = 0.
step = 0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

self.piece_count:  110
dataset_len:  11000
self.piece_count:  110
dataset_len:  11000
cuda:0


In [5]:
def reset_weights(model): # reset the weight every fold
    if isinstance(model, nn.LSTM) or isinstance(model, nn.Linear):
        model.reset_parameters()

In [6]:
class LSTM1(nn.Module):
    def __init__(self, output_dim, input_size, hidden_size, num_layers, seq_length):
        super(LSTM1, self).__init__()
        self.output_dim = output_dim #number of classes
        self.num_layers = num_layers #number of layers
        self.input_size = input_size #input size
        self.hidden_size = hidden_size #hidden state
        self.seq_length = seq_length #sequence length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          num_layers=num_layers, batch_first=True) #lstm
        self.fc_1 =  nn.Linear(hidden_size, output_dim) #fully connected to determine output dim

        self.relu = nn.ReLU()

    def forward(self,x):
        # h0, c0 no time information
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #hidden state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device) #internal state
        # Propagate input through LSTM
        # x is MIDI => [44, 512, 128]

        # hn is final state, run over the sequence length
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
        # hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
        # print("output.shape", output.shape)
        # print("hn.shape", hn.shape)
        # out = self.relu(hn)
        out = self.fc_1(output) #final
        return out
 

In [7]:
# Define the model architecture
input_size = 128 #number of features
hidden_size = 1024 #number of features in hidden state
num_layers = 1 #number of stacked lstm layers
seq_len = 512
output_dim = 115 #number of output classes
batch_size_define = 20#128

# model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
# model = LSTM(embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights).to(device)
# model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
# model.train()
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2)

num_epochs = 240 #10
k_folds = 11
cross_valid_results = {}
torch.manual_seed(42)

avg_loss_list = []
all_loss_list = []
val_loss_per_epoch_list = []

#TODO: important cross val record
val_time_loss_list = []
val_dim_loss_list = []
val_mse_loss_list = []
val_per_split_list = [] #just mse loss


In [8]:
def time_wise_loss_fn(preds, labels):
    '''
    calculate time-wise loss for motion (along the time axis)
    input: labels[batch, time, dimension(joint*xyz)]
    preds[batch, time , dimension(joint*xyz)]
    output: time loss
    '''
    # points_2 = ax.scatter(column(each_frame[30:32], 0), column(each_frame[30:32], 1), column(each_frame[30:32], 2), cmap='jet', marker='o', label='body joint', color = 'blue')
    # points_3 = ax.scatter(column(each_frame[32:34], 0), column(each_frame[32:34], 1), column(each_frame[32:34], 2), cmap='jet', marker='o', label='body joint', color = 'red')
    # [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
    # [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
    # [30, 31], [32, 33],  #instrument
    
    # print("preds.shape", preds.shape)
    # print("labels.shape", labels.shape)
    
    # print("preds[9*3:18*3]", preds[:, :, 9*3:19*3].shape)
    # print("labels[9*3:18*3]", labels[:, :, 9*3:19*3].shape)
    
    select_joint_preds = torch.cat((preds[:, :, 9*3:19*3], preds[:, :, 31*3:35*3]), 2)
    select_joint_labels = torch.cat((labels[:, :, 9*3:19*3], labels[:, :, 31*3:35*3]), 2)
    
    # print("select_joint_preds.shape", select_joint_preds.shape)
    # # print(select_joint_preds)
    # print("select_joint_labels.shape", select_joint_labels.shape)
    
    epsilon = 1e-7
    select_joint_preds = select_joint_preds + epsilon

    labels_transpose = torch.permute(select_joint_labels, (0, 2, 1))#tf.transpose(labels, [0, 2, 1]) # [b, 3, t]
    preds_transpose = torch.permute(select_joint_preds, (0, 2, 1))#tf.transpose(preds, [0, 2, 1]) # [b, 3, t]
    # print("labels_transpose.shape", labels_transpose.shape)
    # print("preds_transpose.shape", preds_transpose.shape)
    # print("labels_transpose[:, :, :, None].shape", labels_transpose[:, :, :, None].shape)
    # print("labels_transpose[:, :, None, :].shape", labels_transpose[:, :, None, :].shape)
    label_diff = labels_transpose[:, :, :, None] - labels_transpose [:, :, None, :] # [b, 3, t, t]
    
    preds_diff = preds_transpose[:, :, :, None] - preds_transpose [:, :, None, :] # [b, 3, t, t]
    # print(preds_diff.shape)
    time_loss = (preds_diff - label_diff)**2 # [b, 3, t, t]
    time_loss_value = time_loss.mean() #float()
    torch.cuda.empty_cache()

    return time_loss_value
    
def dim_wise_loss_fn(preds, labels):
    '''
    calculate dimension-wise loss for motion (along the dimension axis)
    input: labels[batch, time, dimension(joint*xyz)]
    preds[batch, time , dimension(joint*xyz)]
    output: dimension loss
    '''
    select_joint_preds = torch.cat((preds[:, :, 9*3:19*3], preds[:, :, 31*3:35*3]), 2)
    select_joint_labels = torch.cat((labels[:, :, 9*3:19*3], labels[:, :, 31*3:35*3]), 2)

    epsilon = 1e-7
    preds = preds + epsilon
    
    label_diff = select_joint_labels[:, :, :, None] - select_joint_labels[:, :, None, :] # [b, t, 3, 3]
    preds_diff = select_joint_preds[:, :, :, None] - select_joint_preds[:, :, None, :] # [b, t, 3, 3]
    dim_loss = (preds_diff - label_diff)**2 # [b, t, 3, 3]
    dim_loss_value = dim_loss.mean() #float()
    torch.cuda.empty_cache()
    
    return dim_loss_value

In [9]:
# def time_wise_loss_fn(preds, labels):
#     '''
#     calculate time-wise loss for motion (along the time axis)
#     input: labels[batch, time, dimension(joint*xyz)]
#     preds[batch, time , dimension(joint*xyz)]
#     output: time loss
#     '''
#     epsilon = 1e-7
#     preds = preds + epsilon

#     labels_transpose = torch.permute(labels, (0, 2, 1))#tf.transpose(labels, [0, 2, 1]) # [b, 3, t]
#     preds_transpose = torch.permute(preds, (0, 2, 1))#tf.transpose(preds, [0, 2, 1]) # [b, 3, t]
#     # print("labels_transpose.shape", labels_transpose.shape)
#     # print("preds_transpose.shape", preds_transpose.shape)
#     # print("labels_transpose[:, :, :, None].shape", labels_transpose[:, :, :, None].shape)
#     # print("labels_transpose[:, :, None, :].shape", labels_transpose[:, :, None, :].shape)
#     label_diff = labels_transpose[:, :, :, None] - labels_transpose [:, :, None, :] # [b, 3, t, t]
    
#     preds_diff = preds_transpose[:, :, :, None] - preds_transpose [:, :, None, :] # [b, 3, t, t]
#     # print(preds_diff.shape)
#     time_loss = (preds_diff - label_diff)**2 # [b, 3, t, t]
#     time_loss_value = time_loss.mean() #float()
#     torch.cuda.empty_cache()

#     return time_loss_value
    
# def dim_wise_loss_fn(preds, labels):
#     '''
#     calculate dimension-wise loss for motion (along the dimension axis)
#     input: labels[batch, time, dimension(joint*xyz)]
#     preds[batch, time , dimension(joint*xyz)]
#     output: dimension loss
#     '''
#     epsilon = 1e-7
#     preds = preds + epsilon
    
#     label_diff = labels[:, :, :, None] - labels[:, :, None, :] # [b, t, 3, 3]
#     preds_diff = preds[:, :, :, None] - preds[:, :, None, :] # [b, t, 3, 3]
#     dim_loss = (preds_diff - label_diff)**2 # [b, t, 3, 3]
#     dim_loss_value = dim_loss.mean() #float()
#     torch.cuda.empty_cache()
    
#     return dim_loss_value

In [10]:
def customized_mse_loss(output, target):
    # target = target.transpose(0, 1)

    # print("output.shape:", output.shape) #torch.Size([20, 513, 102])
    # print("target.shape:", target.shape) #torch.Size([20, 513, 102])

    w1_time = 0.3
    w2_dim = 0.3
    w3_mse = 0.4

    mse_loss = F.mse_loss(output, target)
    time_loss = time_wise_loss_fn(output, target)
    dim_loss = dim_wise_loss_fn(output, target)

    # print("time_loss:", time_loss)
    # print("dim_loss:", dim_loss)
    # print("mse_loss:", mse_loss)
    val_time_loss_list.append(time_loss.cpu().item())
    val_dim_loss_list.append(dim_loss.cpu().item())
    val_mse_loss_list.append(mse_loss.cpu().item())

    segment_loss = (w1_time * time_loss) + (w2_dim * dim_loss) + (w3_mse * mse_loss)
    torch.cuda.empty_cache()
    return  segment_loss

In [11]:
def evaluate_lstm_cross(model, split_count):
    model.eval()
    print('Validation')
    valid_running_loss = 0.0
    counter = 0
    # previous_output = torch.zeros(512, 102).to(device)
    
    outputs_save = []
    
    np.savetxt('./temp_path.txt', val_data_read[split_count], delimiter="\n", fmt="%s")
    # print(val_data_read[split_count])
    val_dataloader = get_val_dataloader('./temp_path.txt', batch_size=11)
    
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(val_dataloader): #tqdm(enumerate(val_dataloader), total=len(val_dataloader))
            counter += 1

            inputs = inputs.to(device).float()
            targets = targets.to(device).float()
            # print("val inputs.shape:", inputs.shape)
            # print("val targets.shape:", targets.shape)
            outputs = model(inputs)
            # print("val outputs.shape:", outputs.shape)

            loss =  F.mse_loss(outputs, targets)
            valid_running_loss += loss.cpu().item()
            # previous_output = outputs
            outputs_save.append(np.asarray(outputs.cpu()))

    loc_dt = datetime.datetime.today()
    loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
    if not os.path.exists("./output_eval/"):
        os.makedirs('./output_eval/')

    # print("counter", counter)
    # print("val_data_read[split_count][counter]", val_data_read[split_count][counter])
    val_file_name = val_data_read[split_count][counter].split('/')[2].split('.')[0]

    # print("val file_name:", val_file_name)
    # print("outputs_save length: ", len(outputs_save), ", element shape: " , outputs_save[0].shape)
        
    eval_output = open("./output_eval/[split_" + str(split_count) + "][midi_with_anno][total" + str(num_epochs) + "_hs" + str(hidden_size) +"]save_"+ str(loc_dt_format) + "_l1_loss_" + str(loss.cpu().item())+".pkl", 'wb')
    pickle.dump(np.asarray(outputs_save), eval_output)
    eval_output.close()
    
    # print("val counter:", counter)
    epoch_val_loss_f1 = valid_running_loss / counter
    val_per_split_list.append(epoch_val_loss_f1)
    print("split_count:", split_count, ", epoch_val_loss:", epoch_val_loss_f1)
    os.remove("./temp_path.txt")
    return #epoch_val_loss_f1
# model.train()

In [12]:
kf = KFold(n_splits=k_folds)
print(kf.get_n_splits(dataset))
KFold(n_splits=k_folds, random_state=None, shuffle=False)
# for i, (train_index, test_index) in enumerate(kf.split(X)):
#     #...TODO
split_count = 0
random_pick_fold = 6#random.randint(0, 10) #0~10
print("random: ", random_pick_fold)
# for fold,(train_idx, test_idx) in enumerate(kf.split(dataset)): #TODO: random pick 1 fold
for (train_idx, test_idx) in itertools.islice(kf.split(dataset), random_pick_fold, random_pick_fold+1):
    # print('------------fold no---------{}----------------------'.format(fold))
    print('------------fold no---------{}----------------------'.format(random_pick_fold))
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)
    print("train_idx:", train_idx[0], "~", train_idx[-1], " test_idx:", test_idx[0], "~", test_idx[-1])

    train_loader = DataLoader(
                        dataset,
                        num_workers=0,
                        pin_memory=False,
                        drop_last=False,
                        batch_size=batch_size_define, sampler=train_subsampler) #bs=40:4.49G, bs=128:14.65G

    val_loader = DataLoader(
                        dataset,
                        num_workers=0,
                        pin_memory=False,
                        drop_last=False,
                        batch_size=batch_size_define, sampler=val_subsampler)
    
    model = LSTM1(output_dim, input_size, hidden_size, num_layers, seq_len).to(device) #our lstm class
    model.apply(reset_weights)
    
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        print(f'Starting epoch {epoch+1}')
        losses = []
        loss = 0
        mean_loss = 0
        for i, (midi_batch, motion_batch) in enumerate(train_loader):
            model.train()
            
            midi_batch = midi_batch.to(device).float()
            motion_batch = motion_batch.to(device).float()

            optimizer.zero_grad()
            output = model(midi_batch) #midi_batch
            # print("train inputs.shape:", midi_batch.shape, torch.isnan(midi_batch).any())
            # # print(motion_batch)
            # print("train targets.shape:", motion_batch.shape, torch.isnan(motion_batch).any())
            # print("train outputs.shape:", output.shape, torch.isnan(output).any())

            # loss =  F.mse_loss(output, motion_batch)
            # loss = customized_mse_loss(output.cpu(), motion_batch.cpu())
            loss = customized_mse_loss(output, motion_batch)
            
            losses.append(loss.cpu().item()) #.cpu().item()
            all_loss_list.append(loss.cpu().item()) #.cpu().item()
            loss.backward()

            optimizer.step()

            # print(f"Epoch {epoch}, batch {i}: loss = {loss.cpu().item():.6f}") #.cpu().item()

        # print(losses, sum(losses), len(losses))
        mean_loss = sum(losses)/len(losses)
        # correct, total = 0, 0
        valid_running_loss = 0.0
        counter = 0
        with torch.no_grad():
            for i, (midi_test, motion_test) in enumerate(val_loader):
                
                inputs = midi_test.to(device).float()
                targets = motion_test.to(device).float()

                outputs = model(inputs)
                # print("val inputs.shape:", inputs.shape)
                # print("val targets.shape:", targets.shape)
                # print("val outputs.shape:", outputs.shape)

                val_loss =  customized_mse_loss(outputs, targets)
                valid_running_loss += val_loss.cpu().item() #.cpu().item()
                counter += 1
            
            epoch_val_loss = valid_running_loss / counter
            # print(f"Epoch {epoch}: val_loss = {epoch_val_loss:.6f}") #.cpu().item()

        avg_loss_list.append(mean_loss) #.cpu().item()
        val_loss_per_epoch_list.append(epoch_val_loss) #.cpu().item()

        cross_valid_results[0] = epoch_val_loss
        
        loc_dt = datetime.datetime.today()
        loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
        if (epoch+1)%100 == 0:
            torch.save({
                'epoch':epoch,
                'model_state_dict':model.state_dict(),
                'optimizer_state_dict':optimizer.state_dict(),
                'loss':loss
            },  "./model_save/[midi_with_anno][total"+str(num_epochs)+ "_hs" + str(hidden_size) +"]LSTM_save_epoch_" + str(epoch)+ "_"+ str(loc_dt_format) + "_avg_loss_" + str(mean_loss) +".tar")

    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {1} FOLDS')
    print('--------------------------------')
    sum_loss = 0.0
    for key, value in cross_valid_results.items():
        print(f'Fold loss {key}: {value}')
        sum_loss += value
    print(f'Average vaildation new loss: {sum_loss/len(cross_valid_results.items())}')
    
    # validation result save
    evaluate_lstm_cross(model, split_count)
    
    split_count += 1

11
random:  6
------------fold no---------6----------------------
train_idx: 0 ~ 10999  test_idx: 6000 ~ 6999


Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Starting epoch 20
Starting epoch 21
Starting epoch 22
Starting epoch 23
Starting epoch 24
Starting epoch 25
Starting epoch 26
Starting epoch 27
Starting epoch 28
Starting epoch 29
Starting epoch 30
Starting epoch 31
Starting epoch 32
Starting epoch 33
Starting epoch 34
Starting epoch 35
Starting epoch 36
Starting epoch 37
Starting epoch 38
Starting epoch 39
Starting epoch 40
Starting epoch 41
Starting epoch 42
Starting epoch 43
Starting epoch 44
Starting epoch 45
Starting epoch 46
Starting epoch 47
Starting epoch 48
Starting epoch 49
Starting epoch 50
Starting epoch 51
Starting epoch 52
Starting epoch 53
Starting epoch 54
Starting epoch 55
Starting epoch 56
S

In [13]:
print(val_data_read[split_count])

['preprocessed_data_save_cross/midi/vio01_Elgar_S1_T1_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Elgar_S1_T2_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Flower_S1_T1_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Flower_S1_T2_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Mend_S1_T1_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Mend_S1_T2_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Mozart1_S1_T1_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Mozart1_S1_T2_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Mozart2_S1_T1_midi_data.pkl'
 'preprocessed_data_save_cross/midi/vio01_Mozart2_S1_T2_midi_data.pkl']


In [14]:
# # train the model
# for epoch in range(num_epochs):
#     # previous_output = torch.zeros(1, 512, 102).to(device)
#     losses = []
#     for i, (midi_batch, motion_batch) in enumerate(dataloader):
#         model.train()
        
#         midi_batch = midi_batch.to(device).float()
#         motion_batch = motion_batch.to(device).float()
#         # print("midi_batch", midi_batch.shape)
#         # print("motion_batch", motion_batch.shape)

#         optimizer.zero_grad()
#         output = model(midi_batch) #midi_batch
#         # print("output.shape", output.shape)

#         # motion_ground_truth_padding = F.pad(motion_batch, (0,0,0,1), value = 1) #<eot>
        
#         # loss =  F.mse_loss(output, motion_ground_truth_padding)
#         loss =  F.mse_loss(output, motion_batch)
#         # loss = customized_mse_loss(output, motion_ground_truth_padding, previous_output, midi_batch)
#         # loss = customized_mse_loss(output, motion_batch, previous_output, midi_batch)

#         # losses 累計lose
#         losses.append(loss.cpu().item())
#         all_loss_list.append(loss.cpu().item())
#         loss.backward()

#         optimizer.step()
#         mean_loss = sum(losses)/len(losses)

#         print(f"Epoch {epoch}, batch {i}: loss = {loss.cpu().item():.6f}")

#         # scheduler.step(1)
#         # previous_output = output

#         loc_dt = datetime.datetime.today()
#         loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")

#     val_loss = evaluate_lstm(model, val_dataloader) #CUDA out of memory
#     val_loss_per_epoch_list.append(val_loss)
#     print(f"Epoch {epoch}: val_loss = {val_loss:.6f}")
#     # save_best_model(
#     #         val_loss, epoch, model, optimizer, loss, loc_dt_format, mean_loss
#     #     )
#     avg_loss_list.append(mean_loss)
#     loc_dt = datetime.datetime.today()
#     loc_dt_format = loc_dt.strftime("%Y-%m-%d_%H-%M-%S")
#     if (epoch+1)%100 == 0:
#         torch.save({
#             'epoch':epoch,
#             'model_state_dict':model.state_dict(),
#             'optimizer_state_dict':optimizer.state_dict(),
#             'loss':loss
#         }, "./model_save/[100epoch]LSTM_save_epoch_" + str(epoch)+ "_"+ str(loc_dt_format) + "_avg_loss_" + str(mean_loss) +".tar")

In [15]:
print(loc_dt_format)
print(avg_loss_list)

2023-08-29_19-01-28
[0.036506049644201996, 0.028240553986281156, 0.027329270269721748, 0.026914752796292307, 0.0254004777520895, 0.023686511896550657, 0.022424883496016262, 0.021085130911320447, 0.0197055632956326, 0.018583010891452433, 0.017622636176645754, 0.016576052198186517, 0.015558246308937668, 0.014633990848436952, 0.014134552668780088, 0.013024052802473306, 0.01252521170862019, 0.011785013131797313, 0.011420617239549756, 0.010868214699439705, 0.010209055319428444, 0.010627073904499412, 0.009348172187805176, 0.009081989883445203, 0.008668719336390495, 0.008595671903342008, 0.00823026608210057, 0.007650123208761215, 0.007921516943722963, 0.007284213876351714, 0.00776080010086298, 0.006793022966012359, 0.006673933672718704, 0.006424832393415272, 0.00619872965849936, 0.006312788440845906, 0.0062612501485273244, 0.005931016584858298, 0.005776508420705795, 0.005644883054774254, 0.005770286031067371, 0.0055581523478031155, 0.005240004642866552, 0.005392564587295055, 0.005090983828995

In [16]:
print(val_loss_per_epoch_list)

[0.02970727939158678, 0.027741677537560464, 0.02713876258581877, 0.025719397738575935, 0.024386565163731576, 0.02291492436081171, 0.021640875153243543, 0.02014437261968851, 0.018825836256146432, 0.017281305864453315, 0.016669521369040013, 0.015760119650512935, 0.015025778375566005, 0.014142039697617292, 0.013489802181720734, 0.012669011130928992, 0.011863254345953464, 0.011604671142995357, 0.01201988173648715, 0.010126046277582646, 0.010001765377819539, 0.010183407906442881, 0.008866772605106234, 0.008896631449460983, 0.008152917502447963, 0.008488452369347215, 0.0077419568784534935, 0.00764692010357976, 0.007896745596081018, 0.008293641842901706, 0.006884624976664782, 0.006781280739232898, 0.006633356660604477, 0.006270498568192125, 0.006271800482645631, 0.006071901572868228, 0.005905726030468941, 0.005739257242530584, 0.005474960217252373, 0.008189184172078968, 0.0054707796964794395, 0.005522230146452784, 0.005397624345496297, 0.0055448061414062975, 0.004890471524558962, 0.0046936895

In [17]:
# def lr_lambda(epoch):
#     # LR to be 0.1 * (1/1+0.01*epoch)
#     base_lr = 0.1
#     factor = 0.01
#     return base_lr/(1+factor*epoch)

In [18]:
# scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.3, total_iters=10)
# scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [19]:
# val_time_loss_list
# val_dim_loss_list
# val_mse_loss_list
plt.cla()
plt.clf()
print(len(val_time_loss_list))
val_time_loss_list_dataframe = pd.DataFrame(val_time_loss_list)
plt.plot(np.array(val_time_loss_list_dataframe.index), np.array(val_time_loss_list_dataframe[0]))
plt.savefig("avg_time_loss_training.jpg")
plt.show()

plt.cla()
plt.clf()
print(len(val_dim_loss_list))
val_dim_loss_list_dataframe = pd.DataFrame(val_dim_loss_list)
plt.plot(np.array(val_dim_loss_list_dataframe.index), np.array(val_dim_loss_list_dataframe[0]))
plt.savefig("avg_dim_loss_training.jpg")
plt.show()

plt.cla()
plt.clf()
print(len(val_mse_loss_list))
val_mse_loss_list_dataframe = pd.DataFrame(val_mse_loss_list)
plt.plot(np.array(val_mse_loss_list_dataframe.index), np.array(val_mse_loss_list_dataframe[0]))
plt.savefig("avg_mse_loss_training.jpg")
plt.show()

132000
132000
132000


In [20]:
plt.cla()
plt.clf()

In [21]:
print(len(avg_loss_list))
avg_loss_list_dataframe = pd.DataFrame(avg_loss_list)

240


In [22]:
avg_loss_list_dataframe

,0
0,0.036506
1,0.028241
2,0.027329
3,0.026915
4,0.025400
...,...
235,0.002820
236,0.002983
237,0.002944
238,0.002867


In [23]:
plt.plot(np.array(avg_loss_list_dataframe.index), np.array(avg_loss_list_dataframe[0]))
plt.savefig("avg_loss_training.jpg")
plt.show()

In [24]:
plt.cla()
plt.clf()

In [25]:
loss_list_dataframe = pd.DataFrame(all_loss_list)

In [26]:
plt.plot(np.array(loss_list_dataframe.index), np.array(loss_list_dataframe[0]))
plt.savefig("training_loss.jpg")
plt.show()

In [27]:
plt.cla()
plt.clf()

In [28]:
val_loss_per_epoch_list_dataframe = pd.DataFrame(val_loss_per_epoch_list)

In [29]:
plt.plot(np.array(val_loss_per_epoch_list_dataframe.index), np.array(val_loss_per_epoch_list_dataframe[0]))
plt.savefig("training_val_loss.jpg")
plt.show()

In [30]:
def predict(model, input, device):
    model.eval()
    with torch.no_grad():
        input = torch.as_tensor(input).to(torch.float32).to(device)
        # print(target.shape)
        # target = torch.as_tensor(target).to(torch.float32).to(device)
        # TODO: target should be <sos>, should not random
        outputs = model(input)
        return outputs.cpu().numpy()

In [31]:
def read_midi(filename, specific_fps):
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(filename)

    piano_roll = midi_data.get_piano_roll(fs=specific_fps)  # 40fps #250fps
    piano_roll[piano_roll > 0] = 1

    return piano_roll

In [32]:
test_datapath = "./BWV1001/"
change_fps = 40
test_midi_path_list = glob.glob(test_datapath + "*.mid")
test_data_list = []
test_music_list = []
for test_midi in test_midi_path_list:
    str_name = test_midi
    print("str_name:", str_name)
    filename = str_name.split('/')[2]
    filecode = filename.split('.')[0]
    print("filecode: ",filecode)
    test_music_list.append(filecode)
    
    print(test_midi)
    read_piano_roll = read_midi(test_midi, change_fps)
    read_piano_roll_transpose = read_piano_roll.T
    print(read_piano_roll_transpose.shape)
    test_midi_len = read_piano_roll_transpose.shape[0]
    test_data_list.append(read_piano_roll_transpose)

str_name: ./BWV1001/vs1-1ada.mid
filecode:  vs1-1ada
./BWV1001/vs1-1ada.mid
(8171, 128)
str_name: ./BWV1001/vs1-2fug.mid
filecode:  vs1-2fug
./BWV1001/vs1-2fug.mid
(11537, 128)
str_name: ./BWV1001/vs1-3sic.mid
filecode:  vs1-3sic
./BWV1001/vs1-3sic.mid
(6993, 128)
str_name: ./BWV1001/vs1-4prs.mid
filecode:  vs1-4prs
./BWV1001/vs1-4prs.mid
(7897, 128)


In [33]:
def column(matrix, i):
    return [row[i] for row in matrix]

def test_render_animation(fps, output, azim, prediction, ground_truth=None):
    prediction_array = np.asarray(prediction)
    print(prediction_array.size)
    limit = len(prediction_array)
    print("limit", limit)
    size = 6#6
    fps = 40

    # Skeleton layout
    parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                [8, 18], [8, 20], [13, 21], [13, 19],
                [5, 20], [5, 21], [7, 18], [7, 19],
                [18, 19], [19, 21], [21, 20], [20, 18], #waist
                [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
    # joints_right = [1, 2, 12, 13, 14]

    prediction_array[:, :, 2] += 0.1 #[:, :, 2]
    if ground_truth is not None:
        ground_truth[:, :, 2] += 0.1
        poses = {'Prediction': prediction_array,
                 'Ground_truth': ground_truth}
    else:
        poses = {'Prediction': prediction_array}
    

    fig = plt.figure()#(figsize=(size*len(poses), size))
    # ax_3d = []
    # lines_3d = []
    radius = 1#14 #3.7#
    # print(poses)
    for index, (title, data) in enumerate(poses.items()):
        ax = fig.add_subplot(1, len(poses), index + 1, projection='3d')
        ax.clear()
        print(data)
        ims = [] #每一 frame 都存
        for frame_index, each_frame in enumerate(data):
            # print("each_frame")
            # print(each_frame)
            ax.view_init(elev=15., azim=azim)
            ax.set_xlim3d([-radius/2, radius/2])
            ax.set_zlim3d([0, radius])
            ax.set_ylim3d([-radius/2, radius/2])
            ax.set_aspect('auto') #ax.set_aspect('equal')

            # print(title)
            points = ax.scatter(column(each_frame[:30], 0), column(each_frame[:30], 1), column(each_frame[:30], 2), cmap='jet', marker='o', label='body joint', color = 'black')
            points_2 = ax.scatter(column(each_frame[30:32], 0), column(each_frame[30:32], 1), column(each_frame[30:32], 2), cmap='jet', marker='o', label='body joint', color = 'blue')
            points_3 = ax.scatter(column(each_frame[32:34], 0), column(each_frame[32:34], 1), column(each_frame[32:34], 2), cmap='jet', marker='o', label='body joint', color = 'red')
            
            # ax.scatter(column(each_frame, 0), column(each_frame, 1), column(each_frame, 2), cmap='jet', marker='o', label='body joint')
            # ax.legend()
            # print("+++")
            
            parents = [[0, 1], [1, 3], [3, 2], [0, 2],#head
                        [8, 6], [6, 13], [13, 4], [4, 8],#shoulder
                        [6, 4], [4, 5], [5, 7], [7, 6],#Upper torso
                        [8, 18], [8, 20], [13, 21], [13, 19],
                        [5, 20], [5, 21], [7, 18], [7, 19],
                        [18, 19], [19, 21], [21, 20], [20, 18], #waist
                        [18, 22], [20, 22], [22, 23], [22, 25], [23, 25], [24,23], [24, 25],  #right lag
                        [21, 26], [19, 26], [26, 27], [26, 29], [27, 29], [28, 27], [28, 29], #left lag
                        [8, 9], [9, 11], [9, 10], [10, 11], [10, 12], [9, 12], [11, 12], #right hand
                        [13, 14], [14, 16], [14, 15], [16, 15], [14, 17], [16, 17], [15, 17], #left hand
                        [30, 31], [32, 33],  #instrument
                        # [31, 33], [30, 32], [30, 31], [32, 33], [31, 32], [30, 33] #instrument
                        ]
            lines = []
            # draw line
            
            # lines = [ax.plot([each_frame[vs][0], each_frame[ve][0]],
            #                  [each_frame[vs][1], each_frame[ve][1]],
            #                  [each_frame[vs][2], each_frame[ve][2]]) for (vs, ve) in parents]
            line_num = len(parents)
            for idx, each_line in enumerate(parents):
                vec_start = each_frame[each_line[0]]
                vec_end = each_frame[each_line[1]]
                # print(vec_start)
                # print(vec_end)
                line_color = "black"
                if idx == line_num-2:
                    line_color = "blue"
                if idx == line_num-1:
                    line_color = "red"
                # ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]])
                
                temp, = ax.plot([vec_start[0], vec_end[0]], [vec_start[1], vec_end[1]], [vec_start[2], vec_end[2]], color=line_color)
                lines.append(temp)

            # ax.figure.savefig('./test_pic/pic' + str(frame_index) + '.png', dpi=100, bbox_inches = 'tight')

            # ims.append([points])
            # image_frame = [points].extend(lines)
            ims.append([points]+[points_2]+[points_3]+lines) #TODO: try extend

            # plt.cla()
            # print("+++")

    anim = matplotlib.animation.ArtistAnimation(fig, ims, interval=1000/fps)

    if output.endswith('.mp4'):
        FFwriter = matplotlib.animation.FFMpegWriter(fps=fps, extra_args=['-vcodec', 'libx264'])
        anim.save(output, writer=FFwriter)
    elif output.endswith('.gif'):
        anim.save(output, fps=fps, dpi=100, writer='imagemagick')
    else:
        raise ValueError('Unsupported output format (only .mp4 and .gif are supported)')

In [34]:
def plot(audio_path, plot_path, prediction, sample_time, fps, name=""): #audio_path, plot_path, 
    # render_animation(fps, output='new_temp.mp4', azim=75, prediction=prediction)
    test_render_animation(fps, output='new_temp_' + name + '.mp4', azim=75, prediction=prediction)

    # # #merge with wav
    input_video = ffmpeg.input('new_temp_' + name + '.mp4')
    fluid_syn = FluidSynth()
    fluid_syn.midi_to_audio(audio_path, './output' + name + '.wav')
    input_audio = ffmpeg.input('./output' + name + '.wav')
    # output = ffmpeg.output(video, audio, plot_path, vcodec='copy', acodec='aac', strict='experimental')
    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(plot_path).run()
    # os.remove('new_temp_' + name + '.mp4')

In [35]:
model.eval()

full_prediction = pd.DataFrame()
num_count = 0
# read midi
# test_dataloader = get_dataloader(test_datapath, batch_size=1)
for test_batch in test_data_list:
    with torch.no_grad():
        # first_target = torch.zeros(test_batch.shape[0],112)
        # print(first_target.shape)
        test_input = test_batch[None, :]
        # test_target = first_target[None, :]
        print("test_input", test_input.shape)
        # print("test_target", test_target.shape)
        prediction = predict(model, test_input, device)
        
        # print(prediction.shape)
        
        prediction  = prediction[:, :, :102]
        print("prediction.shape", prediction.shape)
        
        # full_prediction.append(prediction)
        full_prediction = pd.DataFrame(prediction[0])
        print("full_prediction", full_prediction.shape)
        
        # prev_prediction = prediction[0][:-1][None, :]
        # print(prev_prediction.shape)
        
        Row_list_prediction =[]
        
        filecode = test_music_list[num_count]
    
        # Iterate over each row
        for index, rows in full_prediction.iterrows():
            #fill nan
            rows = rows.fillna(0)
            # Create list for the current row
            my_list = rows.values.tolist()
            # print(my_list)
            
            my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
            # append the list to the final list
            Row_list_prediction.append(my_list_per3)

        # print(len(Row_list_prediction), len(Row_list_prediction[0]),len(Row_list_prediction[0][0]))
        plot(test_datapath + test_music_list[num_count] + ".mid", "./video_" + filecode + "_test_predict.mp4", Row_list_prediction[:800], None, 40, filecode) #ow_list[0:900]
        # print("prediction.shape", prediction.shape)
        prediction_arr = np.array(Row_list_prediction)
        # formated_motion = prediction_format(full_prediction)
        # # # plot(formated_motion)
        # audio_path = test_music_list[num_count][0]
        # output_path = "test_output_" + filecode + ".mp4"
        # plot(formated_motion, audio_path, output_path, None, 10, filecode)
        num_count += 1

# model.train()

test_input (1, 8171, 128)
prediction.shape (1, 8171, 102)
full_prediction (8171, 102)


81600
limit 800
[[[ 0.06707532  0.14623378  1.09799663]
  [ 0.00258634  0.10184368  1.11649439]
  [ 0.11204418  0.07453063  1.09259913]
  ...
  [ 0.01823345  0.10455879  0.98583308]
  [-0.12736011  0.10543215  1.09079823]
  [ 0.07680516  0.2495077   0.70757035]]

 [[ 0.05115314  0.13699211  1.09537325]
  [-0.01475676  0.09039481  1.11393878]
  [ 0.09782651  0.06581026  1.09846298]
  ...
  [ 0.01103011  0.09342705  0.98862795]
  [-0.12503298  0.10632301  1.11022065]
  [ 0.0719573   0.23709954  0.70436189]]

 [[ 0.07804805  0.1454045   1.09611962]
  [ 0.01184683  0.09904011  1.1249927 ]
  [ 0.12583956  0.07348096  1.10471282]
  ...
  [ 0.03153365  0.0999698   0.99900631]
  [-0.10470776  0.11462727  1.14229343]
  [ 0.07498477  0.232903    0.70422105]]

 ...

 [[ 0.0628603   0.07972923  1.09730939]
  [ 0.00303803  0.0315951   1.11151764]
  [ 0.11470178  0.01147797  1.08748249]
  ...
  [ 0.01159235  0.06912196  0.969537  ]
  [-0.12685555  0.05645965  1.05633048]
  [ 0.10148378  0.25212052  

fluidsynth: panic: An error occurred while reading from stdin.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file './outputvs1-1ada.wav'..


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

...

ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

test_input (1, 7897, 128)
prediction.shape (1, 7897, 102)
full_prediction (7897, 102)
81600
limit 800
[[[ 0.03202517  0.13172574  1.09443752]
  [-0.03247565  0.08626537  1.11164687]
  [ 0.08032922  0.06376071  1.09919349]
  ...
  [-0.00252285  0.09604838  0.98264906]
  [-0.13655286  0.11343112  1.13013456]
  [ 0.0307449   0.20009252  0.7321708 ]]

 [[ 0.03904887  0.12113222  1.09817145]
  [-0.02431603  0.07583935  1.11001227]
  [ 0.08708217  0.0524838   1.09690932]
  ...
  [-0.00194915  0.09720737  0.97738675]
  [-0.14096889  0.09648541  1.11000011]
  [ 0.05867606  0.2166611   0.72827671]]

 [[ 0.04718716  0.12062903  1.09737513]
  [-0.01639954  0.07675069  1.11078129]
  [ 0.09329867  0.05070091  1.09303955]
  ...
  [ 0.00176183  0.09654571  0.97846428]
  [-0.14288019  0.0928546   1.12655029]
  [ 0.05280043  0.21714219  0.73542348]]

 ...

 [[ 0.04415157  0.13033836  1.10038743]
  [-0.01782994  0.08831846  1.11905143]
  [ 0.09053717  0.06274089  1.09927813]
  ...
  [-0.00391879  0.1001

fluidsynth: panic: An error occurred while reading from stdin.


FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file './outputvs1-4prs.wav'..


ffmpeg version 4.2.2 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7.3.0 (crosstool-NG 1.23.0.449-a04d0)
  configuration: --prefix=/home/ilc/anaconda3/envs/sinica --cc=/tmp/build/80754af9/ffmpeg_1587154242452/_build_env/bin/x86_64-conda_cos6-linux-gnu-cc --disable-doc --enable-avresample --enable-gmp --enable-hardcoded-tables --enable-libfreetype --enable-libvpx --enable-pthreads --enable-libopus --enable-postproc --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame --disable-nonfree --enable-gpl --enable-gnutls --disable-openssl --enable-libopenh264 --enable-libx264
  libavutil      56. 31.100 / 56. 31.100
  libavcodec     58. 54.100 / 58. 54.100
  libavformat    58. 29.100 / 58. 29.100
  libavdevice    58.  8.100 / 58.  8.100
  libavfilter     7. 57.100 /  7. 57.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  5.100 /  5.  5.100
  libswresample   3.  5.100 /  3.  5.100
  libpostproc    55

In [36]:
model.eval()

full_prediction = pd.DataFrame()
num_count = 0
# read midi
# test_dataloader = get_dataloader(test_datapath, batch_size=1)
for test_batch in test_data_list:
    with torch.no_grad():
        first_target = torch.zeros(test_batch.shape[0],115)
        print(first_target.shape)
        test_input = test_batch[None, :]
        test_target = first_target[None, :]
        print("test_input", test_input.shape)
        print("test_target", test_target.shape)
        prediction = predict(model, test_input, device)
        
        # print(prediction.shape)
        
        prediction  = prediction[:, :, :102]
        print("prediction.shape", prediction.shape)
        
        # full_prediction.append(prediction)
        full_prediction = pd.DataFrame(prediction[0])
        print("full_prediction", full_prediction.shape)
        
        # prev_prediction = prediction[0][:-1][None, :]
        # print(prev_prediction.shape)
        
        Row_list_prediction =[]
        
        filecode = test_music_list[num_count]
    
        # Iterate over each row
        for index, rows in full_prediction.iterrows():
            #fill nan
            rows = rows.fillna(0)
            # Create list for the current row
            my_list = rows.values.tolist()
            # print(my_list)
            
            my_list_per3 = [my_list[i:i+3] for i in range(0, len(my_list), 3)]
            # append the list to the final list
            Row_list_prediction.append(my_list_per3)

        prediction_arr = np.array(Row_list_prediction)
        if not os.path.exists('./output_prediction/[midi_with_anno]'+str(num_layers)+'LSTM_hidden'+str(hidden_size)+'_'+str(num_epochs)+'epoch/'):
            os.makedirs('./output_prediction/[midi_with_anno]'+str(num_layers)+'LSTM_hidden'+str(hidden_size)+'_'+str(num_epochs)+'epoch/')
        midi_data_output = open('./output_prediction/[midi_with_anno]'+str(num_layers)+'LSTM_hidden'+str(hidden_size)+'_'+str(num_epochs)+'epoch/prediction_'+
                                filecode +'.pkl', 'wb')
        pickle.dump(prediction_arr, midi_data_output)
        midi_data_output.close()
        
        num_count += 1

# model.train()

torch.Size([8171, 115])
test_input (1, 8171, 128)
test_target torch.Size([1, 8171, 115])
prediction.shape (1, 8171, 102)
full_prediction (8171, 102)
torch.Size([11537, 115])
test_input (1, 11537, 128)
test_target torch.Size([1, 11537, 115])
prediction.shape (1, 11537, 102)
full_prediction (11537, 102)
torch.Size([6993, 115])
test_input (1, 6993, 128)
test_target torch.Size([1, 6993, 115])
prediction.shape (1, 6993, 102)
full_prediction (6993, 102)
torch.Size([7897, 115])
test_input (1, 7897, 128)
test_target torch.Size([1, 7897, 115])
prediction.shape (1, 7897, 102)
full_prediction (7897, 102)


In [37]:
# final_val_loss = evaluate_lstm(model, val_dataloader)

In [38]:
# print(final_val_loss)